In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as ltb
from sklearn.neural_network import MLPClassifier
from copy import deepcopy
import warnings
import math


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

warnings.filterwarnings('ignore')

# Index variables

In [3]:
subject_number = 1

# Find where the static data end from data file 'SXXX_dynamic.csv'

In [4]:
static_end = []

for i in tqdm(range(10,10+subject_number)):
    current_dynamic = pd.read_csv('data_all_intact/S0' + str(i) + '_dynamic.csv', header=None)

    for j in range(len(current_dynamic)-1, -1, -1):
        if current_dynamic.iloc[j,0] == 0:
            static_end.append(j)
            break
    

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
static_end

[5413137]

# Analysis performance for different features

In [6]:
# save accuracy in array 'acc_features'
# save features name in array 'name_features'
acc_features = []
name_features = []

In [7]:
# STD, RMS, IEMG, MAV, WL, LOG, SSI, SKW, KURT, AAC, DASDV

## STD--Standard deviation

In [ ]:
# index variable
'''change name, features'''
name = 'STD'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in range(len(current_static_grasprepetition)):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(trainx_raw.iloc[a-385:a+1, b].std())
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, b])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw))):
        current_row = []
        for b in range(0,12):
            current_row.append(testx_raw.iloc[a-385:a+1,b].std())
        testx.append(current_row)
        testy.append(trainy_raw.iloc[a, b])